In [ ]:
# default_exp distill.distillation_callback

# Knowledge Distillation

> Train a network in a teacher-student fashion

In [ ]:
# all_slow

In [ ]:
#hide
from nbdev.showdoc import *

%config InlineBackend.figure_format = 'retina'

In [ ]:
#export
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(64))

In [ ]:
#export
class KnowledgeDistillation(Callback):
    def __init__(self, teacher, T:float=20., α:float=0.7):
        store_attr()
    
    def after_loss(self):
        self.teacher.model.eval()
        teacher_output = self.teacher.model(self.x)
        new_loss = DistillationLoss(self.pred, self.y, teacher_output, self.T, self.α)
        self.learn.loss_grad = new_loss

def DistillationLoss(y, labels, teacher_scores, T, alpha):
    return nn.KLDivLoss(reduction='batchmean')(F.log_softmax(y/T, dim=-1), F.softmax(teacher_scores/T, dim=-1)) * (T*T * 2.0 * alpha) + F.cross_entropy(y, labels) * (1. - alpha)

In [ ]:
teacher = cnn_learner(dls, resnet34, metrics=accuracy)
teacher.unfreeze()

In [ ]:
teacher.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.641291,0.531456,0.830853,00:15
1,0.344991,0.279878,0.899188,00:14
2,0.199024,0.202250,0.918809,00:14
3,0.116771,0.169068,0.941137,00:14
4,0.058538,0.159720,0.945196,00:15


## Without KD

In [ ]:
student = Learner(dls, resnet18(num_classes=2), metrics=accuracy)

In [ ]:
student.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.609657,0.598150,0.686062,00:12
1,0.549790,0.527079,0.734100,00:15
2,0.459070,0.471119,0.771313,00:14


## With KD

In [ ]:
student = Learner(dls, resnet18(num_classes=2), metrics=accuracy)

In [ ]:
kd = KnowledgeDistillation(teacher)

In [ ]:
student.fit_one_cycle(3, cbs=kd)

epoch,train_loss,valid_loss,accuracy,time
0,0.791703,0.751935,0.719215,00:15
1,0.771823,0.641491,0.748985,00:16
2,0.666328,0.693159,0.786874,00:16
